In [1]:
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import pandas as pd
import plotly.express as px

# Load the data using pandas
data = pd.read_csv('https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DV0101EN-SkillsNetwork/Data%20Files/historical_automobile_sales.csv')

# Rename the columns with the provided names
data.columns = [
    'Date', 'Year', 'Month', 'Recession', 'Consumer_Confidence',
    'Seasonality_Weight', 'Price', 'Advertising_Expenditure', 'Competition',
    'GDP', 'Growth_Rate', 'Unemployment_Rate', 'Automobile_Sales',
    'Vehicle_Type', 'City'
]

# Initialize the Dash app
app = dash.Dash(__name__)

# Create the dropdown menu options
dropdown_options = [
    {'label': 'Yearly Statistics', 'value': 'Yearly Statistics'},
    {'label': 'Recession Period Statistics', 'value': 'Recession Period Statistics'}
]

year_list = [str(i) for i in range(1980, 2024, 1)]

# Create the layout of the app
app.layout = html.Div([
    html.H1(
        "Automobile Sales Statistics Dashboard",
        style={
            'text-align': 'center',
            'color': '#503D36',
            'font-size': '24px'
        }
    ),
        html.Div([
        html.Label("Select Statistics:"),
        dcc.Dropdown(
            id='dropdown-statistics',
            options=dropdown_options,
            value='Select Statistics',
            placeholder='Select a report type',
            style={
                'width': '80%',
                'padding': '3px',
                'font-size': '20px',
                'text-align-last': 'center'
            }
        ),
    ]),
    html.Div([
        html.Label("Select Year:"),
        dcc.Dropdown(
            id='select-year',
            options=[{'label': year, 'value': year} for year in year_list],
            value='1980',
            disabled=True  
        ),
    ]),
    html.Div(
        id='output-container',
        className='chart-grid',
        style={'display': 'flex'}
    ),
])

@app.callback(
    Output('select-year', 'disabled'),
    Input('dropdown-statistics', 'value')
)
def enable_disable_input_container(selected_statistics):
    return selected_statistics != 'Yearly Statistics'

# Callback for plotting
@app.callback(
    Output('output-container', 'children'),
    Input('dropdown-statistics', 'value'),
    Input('select-year', 'value')
)
def update_output_container(selected_statistics, input_year):
    if selected_statistics == 'Recession Period Statistics':
     
        recession_data = data[data['Recession'] == 1]

        yearly_rec = recession_data.groupby('Year')['Automobile_Sales'].mean().reset_index()
        R_chart1 = dcc.Graph(
            figure=px.line(
                yearly_rec,
                x='Year',
                y='Automobile_Sales',
                title="Average Automobile Sales fluctuation over Recession Period"
            )
        )

        average_sales = recession_data.groupby('Vehicle_Type')['Automobile_Sales'].mean().reset_index()
        R_chart2 = dcc.Graph(
            figure=px.bar(
                average_sales,
                x='Vehicle_Type',
                y='Automobile_Sales',
                title="Average Vehicles Sold by Vehicle Type during Recession"
            )
        )

        exp_rec = recession_data.groupby('Vehicle_Type')['Advertising_Expenditure'].sum().reset_index()
        R_chart3 = dcc.Graph(
            figure=px.pie(
                exp_rec,
                values='Advertising_Expenditure',
                names='Vehicle_Type',
                title="Total Expenditure Share by Vehicle Type during Recession"
            )
        )

        
        unemployment_data = recession_data.groupby(['Year', 'Vehicle_Type'])['Unemployment_Rate'].mean().reset_index()
        R_chart4 = dcc.Graph(
            figure=px.bar(
                unemployment_data,
                x='Year',
                y='Unemployment_Rate',
                color='Vehicle_Type',
                title="Effect of Unemployment Rate on Vehicle Type and Sales during Recession"
            )
        )

        return [
            html.Div(className='chart-item', children=[html.Div(children=R_chart1), html.Div(children=R_chart2)]),
            html.Div(className='chart-item', children=[html.Div(children=R_chart3), html.Div(children=R_chart4)])
        ]
    elif selected_statistics == 'Yearly Statistics':

        yearly_data = data[data['Year'] == int(input_year)]

        yearly_sales = yearly_data.groupby('Month')['Automobile_Sales'].sum().reset_index()
        Y_chart1 = dcc.Graph(
            figure=px.line(
                yearly_sales,
                x='Month',
                y='Automobile_Sales',
                title=f"Monthly Automobile Sales in {input_year}"
            )
        )

        average_price = yearly_data.groupby('Month')['Price'].mean().reset_index()
        Y_chart2 = dcc.Graph(
            figure=px.line(
                average_price,
                x='Month',
                y='Price',
                title=f"Average Vehicle Price in {input_year}"
            )
        )

        vehicle_sales = yearly_data.groupby('Vehicle_Type')['Automobile_Sales'].sum().reset_index()
        Y_chart3 = dcc.Graph(
            figure=px.bar(
                vehicle_sales,
                x='Vehicle_Type',
                y='Automobile_Sales',
                title=f"Vehicle Sales by Type in {input_year}"
            )
        )

        advertising_data = yearly_data.groupby('Vehicle_Type')['Advertising_Expenditure'].sum().reset_index()
        Y_chart4 = dcc.Graph(
            figure=px.pie(
                advertising_data,
                values='Advertising_Expenditure',
                names='Vehicle_Type',
                title=f"Advertising Expenditure Distribution by Vehicle Type in {input_year}"
            )
        )

        return [
            html.Div(className='chart-item', children=[html.Div(children=Y_chart1), html.Div(children=Y_chart2)]),
            html.Div(className='chart-item', children=[html.Div(children=Y_chart3), html.Div(children=Y_chart4)])
        ]


# Run the Dash app
if __name__ == '__main__':
    app.run_server(debug=True)
